# 공용 함수
- 수정하지 않는 것을 추천드립니다

In [1]:
from jy import get_all, get_scaled
import pandas as pd
from neuralprophet import NeuralProphet

In [ ]:
# 상대경로 지정 없이 원본 csv 임포트
main_df = get_all()
scaled_df = get_scaled(0) #0 = sd, 1 = nm, 2 = mm, 3 = rb
main_df_columns = main_df.columns
date_df = main_df.loc[:,"date"]

In [ ]:
# 각 지역구와 관련된 변수들
# common은 지역 구분 없는 전체 변수
gw_vars = main_df.loc[:,main_df_columns[2:9]]
ddm_vars = main_df.loc[:,main_df_columns[9:16]]
sd_vars = main_df.loc[:,main_df_columns[16:23]]
jl_vars = main_df.loc[:,main_df_columns[23:30]]
common_vars = main_df.loc[:,main_df_columns[30:46]]

In [ ]:
# 각 지역구의 따릉이 수요량 
gw_demand=main_df.loc[:,main_df_columns[46]]
ddm_demand=main_df.loc[:,main_df_columns[47]]
sd_demand=main_df.loc[:,main_df_columns[48]]
jl_demand=main_df.loc[:,main_df_columns[49]]

In [ ]:
# 날짜와 수요량 컬럼을 원하는 대로 정리해주는 함수
def create_y (df_demand, date_name, y_name):
    result = pd.concat([date_df, df_demand], axis=1)
    result = result.rename(columns={"date":date_name, result.columns[1]:y_name})
    return result

In [ ]:
# 변수들을 일부 샘플링한 dataframe의 리스트를 반환하는 함수
## amount는 (수요량 이외에) 추가로 샘플링할 변수의 개수 (0, 1, 2 가능. 0은 수요량만 반환)
## df_y는 날짜와 수요량이 포함된 데이터프레임. create_y 함수로 정제한 값을 넣으면 됩니다
## df_list는 추가 변수로 사용할 모든 데이터프레임들의 리스트. ~~~_vars 중 원하는 것을 리스트로 넣으면 됩니다
    ## 예시) 광진구 + 공용 변수의 경우: df_list=[gw_vars, comon_vars]
## 반환하는 값은 튜플 {"data": 데이터프레임, "name":포함된 컬럼명의 리스트} 로 이루어진 리스트입니다
    
def sample_multivariate(amount, df_y, df_list):
    samples = []
    # 데이터프레임을 하나로 통합
    df = pd.concat(df_list, axis=1)
    if amount == 0:
        # y값만 리턴
        samples.append({"data":pd.concat([df_y], axis=1),"name":[]})
    elif amount == 1:
        # sample을 한 column씩 추출
        for index in range(len(df.columns)):
            sample = pd.concat([df_y,df.loc[:,df.columns[index]]],axis=1)
            samples.append({"data":sample, "name":[df.columns[index]]})
    elif amount == 2:
        # sample을 두 column씩 추출
        for first_index in range(len(df.columns)):
            remaining_columns = df.columns[first_index+1:]
            for second_index in range(len(remaining_columns)):
                targets = [df.columns[first_index],remaining_columns[second_index]]
                sample = pd.concat([df_y,df.loc[:,targets]],axis=1)
                samples.append({"data":sample, "name":targets})
    return samples
    

In [ ]:
# 데이터를 자르는 함수
# 일단은 2021년 1월 1일 기준으로 자르게 일반화 해놓았습니다

def split_sample(df):
    train = df.loc[:1095]
    test = df.loc[1096:]
    return {"train":train, "test":test}


In [ ]:
def create_result(indexes):
    return pd.DataFrame(index=["Column"] + indexes)

공용 함수 End